In [ ]:
import math
import os

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

FOLDER = 'tuning_results/40_epochs'
RESULT_PATH = os.path.join(FOLDER, 'tuning_results.csv')

In [ ]:
original_df = pd.read_csv(RESULT_PATH)
print(len(original_df))
original_df.sort_values('val_acc', ascending=False).head()

In [ ]:
def create_norm_column(df, layer_column, batch_column):
    series = pd.Series(data=['' for _ in range(len(df))])
    series[df[layer_column]] = 'layer'
    series[df[batch_column]] = 'batch'
    series[series == ''] = 'none'
    return series

df = original_df.copy()
df['fc_norm'] = create_norm_column(df, 'fc_use_laynorm_inp', 'fc_use_batchnorm_inp')
df['fc_norm'] += ',' + create_norm_column(df, 'fc_use_laynorm', 'fc_use_batchnorm')
df['cnn_norm'] = create_norm_column(df, 'cnn_use_laynorm_inp', 'cnn_use_batchnorm_inp')
df['cnn_norm'] += ',' + create_norm_column(df, 'cnn_use_laynorm', 'cnn_use_batchnorm')

boolean_columns = list(filter(lambda c: '_use_' in c, df.columns))
df = df.drop(columns=boolean_columns)
df = df[[
    'uid',
    'cnn_norm', 'cnn_drop', 'cnn_act',
    'fc_norm', 'fc_lay', 'fc_drop', 'fc_act',
    'lr', 'optimizer',
    'train_acc', 'val_acc'
]]
df['lr'] = df['lr'].apply(math.log10)
df['rel_diff'] = abs(df['train_acc'] - df['val_acc']) / df[['train_acc', 'val_acc']].max(axis=1)
df = df.sort_values('val_acc', ascending=False)

In [ ]:
TOP_COUNT = 10
df.head(TOP_COUNT)

In [ ]:
min_max_df = pd.DataFrame()
for column in ['cnn_drop', 'lr', 'fc_drop']:
    min_max_df['min_' + column] = [df.head(TOP_COUNT)[column].min()]
    min_max_df['max_' + column] = [df.head(TOP_COUNT)[column].max()]
min_max_df

In [ ]:
df.tail(10)

In [ ]:
def draw_loss_acc_plots(row):
    uid = row['uid']
    log_path = os.path.join(FOLDER, uid, 'log.csv')
    plot_df = pd.read_csv(log_path)
    fig, axs = plt.subplots(2, sharex=True)
    fig.suptitle(uid[:8])
    epochs = list(range(1, len(plot_df) + 1))
    for value, ax in zip(['accuracy', 'loss'], axs):
        ax.plot(epochs, plot_df[value], label='train')
        ax.plot(epochs, plot_df['val_{}'.format(value)], label='val')
        ax.set(xlabel='epochs', ylabel=value)
        ax.legend()
    fig.text(1.0, 0.25, str(row[1:]))
    plt.show()


for row in df.head(TOP_COUNT).iloc():
    draw_loss_acc_plots(row)